## Extracting Context Topics and Contexts

In [ ]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict

np.random.seed(42)
with open('../../data/taji_1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [2]:
# Extract topics from context to new column, meanwhile, extracting the context themselves into a list
all_context = []

def extract_topics(context_list):
    topics = []
    for context_item in context_list:
        # element 0 is the topic, element 1 are the contexts
        topics.append(context_item[0])  
        all_context.append(('\n'.join(context_item[1])))  # Collect all contexts
    return topics

df['topics'] = df['context'].apply(extract_topics)

# Get all unique topics and contexts
all_topics = set()

for topics in df['topics']:
    all_topics.update(topics)
    
all_context = list(set(all_context))  # Remove duplicates from contexts
print(f"Total unique contexts: {len(all_context)}")
print(f"Total unique topics: {len(all_topics)}")

Total unique contexts: 470
Total unique topics: 388


In [ ]:
## save all_context to a json file
with open('all_context.json', 'w', encoding='utf-8') as f:
    json.dump(all_context, f, ensure_ascii=False, indent=4)

## Splitting data into Train/Test

Also ensuring the fact that all contexts appear at least once in our train set.

In [6]:
topic_to_samples = defaultdict(list)
for idx, topics in enumerate(df['topics']):
    for topic in topics:
        topic_to_samples[topic].append(idx)

In [7]:
topic_to_samples

defaultdict(list,
            {'۱۳ دلیل برای اینکه ': [0, 1, 2, 256, 257, 258],
             'هانا بیکر ': [0, 1, 2, 510, 511],
             'سامورایی ': [3, 4],
             'سایتو دوسان ': [3, 4],
             'سوپرتاکس ': [5, 6, 246, 247, 248],
             'نینتندو ': [5, 6, 236, 237, 242],
             'مالک سیدیبه ': [7, 8, 9],
             'شیر طلایی ': [7, 8, 9, 241],
             'مسابقه آواز یوروویژن ۲۰۱۹ ': [10],
             'مسابقه آواز یوروویژن ۲۰۱۸ ': [10],
             'نامه فرهنگستان ': [11, 12, 13, 14, 15, 436],
             'سردبیر ': [11, 12, 13, 14, 15],
             'خاطرات بریجت جونز (فیلم) ': [16, 17, 18, 19],
             'جما جونز ': [16, 17, 18, 19],
             'لامپ نئون ': [20, 21, 22, 23],
             'جیوه ': [20, 21, 22, 23, 225, 226, 227, 228, 433, 434, 455],
             'زابیواکا (برنامه تلویزیونی) ': [24, 25, 26, 27, 28, 29, 30, 31],
             'جام جهانی فوتبال ۲۰۱۸ ': [24, 25, 26, 27, 28, 29, 30, 31],
             'استفانی مک\u200cمن ': [32, 3

In [8]:
# one sample per topic for training (to guarantee coverage)
guaranteed_train_indices = set()
for topic, sample_indices in topic_to_samples.items():
    selected_idx = np.random.choice(sample_indices)
    guaranteed_train_indices.add(selected_idx)

print(f"Guaranteed training samples: {len(guaranteed_train_indices)}")

Guaranteed training samples: 280


In [9]:
remaining_indices = set(range(len(df))) - guaranteed_train_indices
remaining_indices = list(remaining_indices)

additional_train_needed = 400 - len(guaranteed_train_indices)

additional_train_indices = np.random.choice(
    remaining_indices, 
    size=additional_train_needed, 
    replace=False
)
train_indices = list(guaranteed_train_indices) + list(additional_train_indices)
all_indices = set(range(len(df)))
test_indices = list(all_indices - set(train_indices))

# Create train and test splits
train_df = df.iloc[train_indices].reset_index(drop=True)
test_df = df.iloc[test_indices].reset_index(drop=True)

# Remove the helper 'topics' column
train_df = train_df.drop('topics', axis=1)
test_df = test_df.drop('topics', axis=1)

print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

Train size: 400
Test size: 152


In [ ]:
# Verify topic coverage in training 
train_topics = set()
for _, row in train_df.iterrows():
    for context_item in row['context']:
        if len(context_item) > 0:
            train_topics.add(context_item[0])

print(f"Topics in training set: {len(train_topics)}")

## Save the splits
# train_df.to_json('train_data.json', orient='records', force_ascii=False, indent=2)
# test_df.to_json('test_data.json', orient='records', force_ascii=False, indent=2)

Topics in training set: 388


# Extracting Knowledge Graph Triplets

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List
import dotenv
import json
import os

In [2]:
dotenv.load_dotenv()

MODEL_NAME = "gpt-4o-mini"

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0,
    api_key=os.getenv("METIS_API_KEY"),
    base_url="https://api.metisai.ir/openai/v1",
)

In [ ]:
# load all_context from json file
with open('all_context.json', 'r', encoding='utf-8') as f:
    all_context = json.load(f)

# print 3 first contexts
print("First 3 contexts:")
for context in all_context[:3]:
    print(context)
    print("-" * 10)

First 3 contexts:
آفریقای جنوبی با نام رسمی جمهوری آفریقای جنوبی  کشوری در جنوب آفریقا و در سواحل دو اقیانوس اطلس و هند است.
این کشور با جمعیتی برابر ۵۹٫۶ میلیون تن چهارمین کشور پر جمعیت در آفریقای زیرصحرا می‌باشد.
----------
مانجاری فاندیس مدل و هنرپیشه اهل هند است.
مانجاری در سال ۲۰۰۸ با بازی در سریال اگه بدونی یا نه شناخته شد.
آقای تقلب فیلم دیگریست که او در آن نقش آفرینی کرده‌است.
----------
لوک اسکای‌واکر شخصیتی تخیلی و قهرمان داستان در سه‌گانهٔ اصلی فرنچایز جنگ ستارگان است که توسط جرج لوکاس پدید آمده‌است.
لوک را مارک همیل به تصویر کشیده‌است و نخستین بار در جنگ ستارگان ظاهر شد و در فیلم‌های امپراتوری ضربه متقابل می‌زند و بازگشت جدای بازگشت.
مارک همیل سه دهه بعد نقش لوک را در سه‌گانهٔ دنبالهٔ این فرنچایز متشکل از فیلم‌های نیرو برمی‌خیزد ، آخرین جدای و خیزش اسکای‌واکر ایفا کرد.
همیل همچنین به‌صورت بدل و دیجیتالی جوان‌شده در قسمت ۱۶ مجموعهٔ مندلورین در نقش لوک حضور داشت.
----------


In [ ]:
class NamedEntities(BaseModel):
    """A structured list of named entities."""
    named_entities: List[str] = Field(
        ...,
        description="A list of named entities extracted from the provided passage."
    )

class KnowledgeTriple(BaseModel):
    """Represents a single Subject-Predicate-Object relationship."""
    subject: str = Field(..., description="The subject of the relationship.")
    predicate: str = Field(..., description="The predicate or verb phrase describing the relationship.")
    object: str = Field(..., description="The object of the relationship.")

class Triples(BaseModel):
    """A structured list of knowledge triples."""
    triples: List[KnowledgeTriple] = Field(
        ...,
        description="A list of knowledge triples (subject, predicate, object) extracted from the passage."
    )

In [11]:
ner_system_prompt = """
Instruction:
Your task is to extract named entities from the given paragraph in the user's message.
Respond with a JSON list of entities.

One-Shot Demonstration:
If the user provides the paragraph:
"Radio City is India's first private FM radio station and was started on 3 July 2001. It plays Hindi, English and regional songs. Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features."

Your output should be a JSON object with a list of the extracted entities:
{{"named_entities": ["Radio City", "India", "3 July 2001", "Hindi", "English", "May 2008", "PlanetRadiocity.com"]}}
"""

# System Prompt for Open Information Extraction (OpenIE)
# This example is taken from Figure 9 of the HippoRAG paper.
openie_system_prompt = """
Instruction:
Your task is to construct an RDF (Resource Description Framework) graph from the given passages and named entity lists.
Respond with a JSON list of triples, with each triple representing a relationship in the RDF graph.
Pay attention to the following requirements:
- Each triple should contain at least one, but preferably two, of the named entities in the list for each passage.
- Clearly resolve pronouns to their specific names to maintain clarity.

One-Shot Demonstration:
If the user provides the paragraph and entity list:
Paragraph:
"Radio City is India's first private FM radio station and was started on 3 July 2001. It plays Hindi, English and regional songs. Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features."
Named Entity List:
["Radio City", "India", "3 July 2001", "Hindi", "English", "May 2008", "PlanetRadiocity.com"]

Your output should be a JSON object containing a list of the extracted triples:
{{"triples": [
    ["Radio City", "located in", "India"],
    ["Radio City", "is", "private FM radio station"],
    ["Radio City", "started on", "3 July 2001"],
    ["Radio City", "plays songs in", "Hindi"],
    ["Radio City", "plays songs in", "English"],
    ["Radio City", "forayed into", "New Media"],
    ["Radio City", "launched", "PlanetRadiocity.com"],
    ["PlanetRadiocity.com", "launched in", "May 2008"],
    ["PlanetRadiocity.com", "is", "music portal"],
    ["PlanetRadiocity.com", "offers", "news"],
    ["PlanetRadiocity.com", "offers", "videos"],
    ["PlanetRadiocity.com", "offers", "songs"]
]}}
"""

In [12]:
ner_prompt = ChatPromptTemplate.from_messages([
    ("system", ner_system_prompt),
    ("human", "{passage}")
])

openie_prompt = ChatPromptTemplate.from_messages([
    ("system", openie_system_prompt),
    ("human", "Paragraph:\n{passage}\n\nNamed Entity List:\n{entities}")
])

In [13]:
structured_ner_llm = llm.with_structured_output(NamedEntities)
structured_openie_llm = llm.with_structured_output(Triples)

ner_chain = ner_prompt | structured_ner_llm
openie_chain = openie_prompt | structured_openie_llm

In [ ]:
def extract_knowledge_structured(passage: str) -> (List[str], List[dict]):
    """
    Implements the two-step HippoRAG extraction process using a system/user prompt structure.
    """
    try:
        ner_response = ner_chain.invoke({"passage": passage})
        entities = ner_response.named_entities
        print(f"  -> Extracted Entities: {entities}")
    except Exception as e:
        print(f"  -> Error during structured NER extraction: {e}")
        return [], []

    if not entities:
        print("  -> No entities found, skipping triple extraction.")
        return [], []

    try:
        entities_str = json.dumps(entities, ensure_ascii=False)
        openie_response = openie_chain.invoke({"passage": passage, "entities": entities_str})
        triples = [triple.model_dump() for triple in openie_response.triples]
        print(f"  -> Extracted Triples: {triples}")
        return entities, triples
    except Exception as e:
        print(f"  -> Error during structured OpenIE extraction: {e}")
        return entities, []

In [23]:
## Test 5 contexts
contexts_to_process = all_context[:5]
knowledge_graph_data = []

print("Starting Knowledge Graph Extraction (HippoRAG Paper Examples)")
for i, context_passage in enumerate(contexts_to_process):
    print(f"\ncontext {i+1}/5:")
    print(f"'{context_passage}'")

    extracted_entities, extracted_triples = extract_knowledge_structured(context_passage)

    if extracted_triples:
        knowledge_graph_data.append({
            "id": i,
            "passage": context_passage,
            "entities": extracted_entities,
            "triples": extracted_triples
        })
    print("-" * 20)

Starting Knowledge Graph Extraction (HippoRAG Paper Examples)

context 1/5:
'آفریقای جنوبی با نام رسمی جمهوری آفریقای جنوبی  کشوری در جنوب آفریقا و در سواحل دو اقیانوس اطلس و هند است.
این کشور با جمعیتی برابر ۵۹٫۶ میلیون تن چهارمین کشور پر جمعیت در آفریقای زیرصحرا می‌باشد.'


--------------------

context 2/5:
'مانجاری فاندیس مدل و هنرپیشه اهل هند است.
مانجاری در سال ۲۰۰۸ با بازی در سریال اگه بدونی یا نه شناخته شد.
آقای تقلب فیلم دیگریست که او در آن نقش آفرینی کرده‌است.'
--------------------

context 3/5:
'لوک اسکای‌واکر شخصیتی تخیلی و قهرمان داستان در سه‌گانهٔ اصلی فرنچایز جنگ ستارگان است که توسط جرج لوکاس پدید آمده‌است.
لوک را مارک همیل به تصویر کشیده‌است و نخستین بار در جنگ ستارگان ظاهر شد و در فیلم‌های امپراتوری ضربه متقابل می‌زند و بازگشت جدای بازگشت.
مارک همیل سه دهه بعد نقش لوک را در سه‌گانهٔ دنبالهٔ این فرنچایز متشکل از فیلم‌های نیرو برمی‌خیزد ، آخرین جدای و خیزش اسکای‌واکر ایفا کرد.
همیل همچنین به‌صورت بدل و دیجیتالی جوان‌شده در قسمت ۱۶ مجموعهٔ مندلورین در نقش لوک حضور داشت.'
--------------------

context 4/5:
'مارشال‌های آمریکایی فیلمی محصول سال ۱۹۹۸ آمریکا به کارگردانی استوارت برد است.
در این فیلم بازیگرانی همچون تامی لی جونز، رابرت داونی جونیور، وسلی اسنایپس، جو پانتولیانو، ایرن ژاکوب، کیت نلیگان، دانیال روبوک، لاتانیا ریچاردسون، پاتریک مالاهید 

In [ ]:
# Now do the same for all of the contexts
from tqdm import tqdm

def extract_knowledge_structured(passage: str) -> (List[str], List[dict]):
    """
    Implements the two-step HippoRAG extraction process using a system/user prompt structure.
    """
    try:
        ner_response = ner_chain.invoke({"passage": passage})
        entities = ner_response.named_entities
        # print(f"  -> Extracted Entities: {entities}")
    except Exception as e:
        print(f"  -> Error during structured NER extraction: {e}")
        return [], []

    if not entities:
        print("  -> No entities found, skipping triple extraction.")
        return [], []

    try:
        entities_str = json.dumps(entities, ensure_ascii=False)
        openie_response = openie_chain.invoke({"passage": passage, "entities": entities_str})
        triples = [triple.model_dump() for triple in openie_response.triples]
        # print(f"  -> Extracted Triples: {triples}")
        return entities, triples
    except Exception as e:
        print(f"  -> Error during structured OpenIE extraction: {e}")
        return entities, []
    
    

knowledge_graph_data = []

print("Starting Knowledge Graph Extraction (HippoRAG Paper Examples)")

i = 0
for context_passage in tqdm(all_context):
    extracted_entities, extracted_triples = extract_knowledge_structured(context_passage)

    if extracted_triples:
        knowledge_graph_data.append({
            "id": i,
            "passage": context_passage,
            "entities": extracted_entities,
            "triples": extracted_triples
        })
        
    i += 1
    print("-" * 20)

Starting Knowledge Graph Extraction (HippoRAG Paper Examples)


  0%|          | 1/470 [00:06<50:50,  6.51s/it]

--------------------


  0%|          | 2/470 [00:11<44:09,  5.66s/it]

--------------------


...

--------------------


 79%|███████▊  | 370/470 [55:35<12:58,  7.79s/it]

--------------------


 79%|███████▊  | 370/470 [55:37<15:02,  9.02s/it]


KeyboardInterrupt: 

In [27]:

def extract_knowledge_structured(passage: str) -> (List[str], List[dict]):
    """
    Implements the two-step HippoRAG extraction process using a system/user prompt structure.
    """
    try:
        ner_response = ner_chain.invoke({"passage": passage})
        entities = ner_response.named_entities
        # print(f"  -> Extracted Entities: {entities}")
    except Exception as e:
        print(f"  -> Error during structured NER extraction: {e}")
        return [], []

    if not entities:
        print("  -> No entities found, skipping triple extraction.")
        return [], []

    try:
        entities_str = json.dumps(entities, ensure_ascii=False)
        openie_response = openie_chain.invoke({"passage": passage, "entities": entities_str})
        triples = [triple.model_dump() for triple in openie_response.triples]
        # print(f"  -> Extracted Triples: {triples}")
        return entities, triples
    except Exception as e:
        print(f"  -> Error during structured OpenIE extraction: {e}")
        return entities, []
    
    

# knowledge_graph_data = []

print("Starting Knowledge Graph Extraction (HippoRAG Paper Examples)")

i = 0
for context_passage in tqdm(all_context[len(knowledge_graph_data):]):
    extracted_entities, extracted_triples = extract_knowledge_structured(context_passage)

    if extracted_triples:
        knowledge_graph_data.append({
            "id": i,
            "passage": context_passage,
            "entities": extracted_entities,
            "triples": extracted_triples
        })
        
    i += 1
    # print("-" * 20)

Starting Knowledge Graph Extraction (HippoRAG Paper Examples)


100%|██████████| 102/102 [14:39<00:00,  8.62s/it]


In [34]:
# save the knowledge graph data to a json file
with open('knowledge_graph_triples_structured.json', 'w', encoding='utf-8') as f:
    json.dump(knowledge_graph_data, f, ensure_ascii=False, indent=4)

# Check prepared data

In [ ]:
import pandas as pd
import igraph as ig
import json
from collections import defaultdict

In [12]:
with open('knowledge_graph_triples_structured.json', 'r', encoding='utf-8') as f:
    knowledge_graph_data = json.load(f)
df_kg = pd.DataFrame(knowledge_graph_data).drop(columns=['id'])
df_kg.head()

,passage,entities,triples
0,آفریقای جنوبی با نام رسمی جمهوری آفریقای جنوبی...,"[آفریقای جنوبی, جمهوری آفریقای جنوبی, جنوب آفر...","[{'subject': 'آفریقای جنوبی', 'predicate': 'ha..."
1,مانجاری فاندیس مدل و هنرپیشه اهل هند است.\nمان...,"[مانجاری فاندیس, هند, ۲۰۰۸, سریال اگه بدونی یا...","[{'subject': 'مانجاری فاندیس', 'predicate': 'i..."
2,لوک اسکای‌واکر شخصیتی تخیلی و قهرمان داستان در...,"[لوک اسکای‌واکر, جرج لوکاس, مارک همیل, جنگ ستا...","[{'subject': 'لوک اسکای‌واکر', 'predicate': 'i..."
3,مارشال‌های آمریکایی فیلمی محصول سال ۱۹۹۸ آمریک...,"[مارشال‌های آمریکایی, ۱۹۹۸, آمریکا, استوارت بر...","[{'subject': 'مارشال‌های آمریکایی', 'predicate..."
4,کیران دسای (Kiran Desai) (زاده ۳ سپتامبر ۱۹۷۱)...,"[کیران دسای, Kiran Desai, ۳ سپتامبر ۱۹۷۱, هند,...","[{'subject': 'کیران دسای', 'predicate': 'is', ..."


In [15]:
all_named_entities = set()
all_triples = []
for entity_list in df_kg['entities']:
    all_named_entities.update(entity_list)
    
for triples in df_kg['triples']:
    for triple in triples:
        all_triples.append((triple['subject'], triple['predicate'], triple['object']))
    
print(f"Total unique named entities: {len(all_named_entities)}")
print(f"Total unique triples: {len(all_triples)}")

Total unique named entities: 3589
Total unique triples: 5694


In [16]:
all_named_entities

{'۱۸ ژانویه ۲۰۲۲',
 'جان کارمک',
 'دهه ۵۰ میلادی',
 'تیم ملی فوتبال فرانسه',
 'روسیه تزاری',
 'پنگوئن',
 '۲۲ اکتبر ۱۷۲۱',
 'اد هلمز',
 'پویانمایی آمریکایی',
 'آیفون ۶اس',
 'Yoshitoshi ABe',
 'بهترین زمان',
 '۱۴۰۰ نفر',
 'سیم های فلزی',
 'علامه',
 'نیو سیکس',
 'ناصر تقوایی',
 '۳ اوت ۱۹۷۱',
 'دوران طلایی اسلامی',
 'سیارات',
 'عربی',
 '۱ بهمن ۱۳۴۱',
 'جرمی ایروین',
 'فلسطین',
 'کامران میرزا',
 'هندوآریایی',
 'هیدروکراکینگ',
 'برادران سوپر اسمش',
 'ژاپنی',
 'کاترین دنو',
 'قهوه',
 'جنوب غربی آسیا',
 'اینفینیتی وارد',
 'فرشته سیاه',
 'برادران وارنر پیکچرز',
 'شصت و سومین دوره مسابقه آواز یوروویژن',
 'محمدعلی نجفی',
 'مارشال\u200cهای آمریکایی',
 'آلبانی',
 'Zosui',
 'زنای با محارم',
 'هواپیمایی ایران ایرتور',
 'دیو گرین',
 'جمهوری خلق چین',
 'جینا فیشر',
 'اتحاد جماهیر شوروی سوسیالیستی',
 'الحَمْد لله',
 'جنگ فالکلند',
 'پدرو آلوارز کابرال',
 '۱۴ مارس ۱۹۳۳',
 'کیران دسای',
 'المپیک لندن',
 'پلیس فراری',
 '۲۰۰۵',
 'آفریقای شمالی',
 'جانوران',
 'جشنواره کن',
 'جسیکا لنگ',
 'قارمون',
 'تار',
 '